In [5]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import statsmodels.api as sm
import scipy
import time
import argparse
import multiprocessing as mp


In [6]:
# Algorithm setting
parser = argparse.ArgumentParser()
parser.add_argument('--grad', '-g', type=str, default='u2g', \
                    help='reinforce, arm, u2g')
parser.add_argument('--lmbd', type=float, default=0.2, \
                    help='Lagrangian parameter, increase with low SNR ')
parser.add_argument('--lr', '-l', type=float, default=0.1, help='base lr')
parser.add_argument('--K', '-k', type=int, default=10, \
                    help='number of MC sample')

# Model setting
parser.add_argument('--name', '-n', default='exp2_fq', help='name')
parser.add_argument('--sigma',  type=float, default=1.414, \
                    help='variance of epsilon, 1.195 or 1.414 for SNR=7 or 5')
parser.add_argument('--N', type=int, default=100, help='number of data')
parser.add_argument('--P', '-b', type=int, default=1000, help='dimension size')

# Experiment setting
parser.add_argument('--iter',  type=int, default=10000, help='maximal iter')
parser.add_argument('--N_test', type=int, default=10, \
                    help='number of repeated experiment')

#args = parser.parse_args()
args, unknown = parser.parse_known_args()
eps = 1e-8

In [7]:
def sigmoid(z):
    return(1/(1+np.exp(-z)))

def F(E,X,Y, lmbd = 5):
    if np.sum(E)>0:
        a1 = np.squeeze(np.where(E != 0))
        X1 = np.take(X, a1, axis=1)
        if len(np.shape(X1))<2:
            X1 = X1[:,None]
        params = scipy.linalg.lstsq(X1, Y,cond=1e-10, lapack_driver='gelsy')[0]
        y_hat = X1.dot(params)
        f_value = np.mean(np.square(Y - y_hat)) + lmbd * np.sum(E)
    else:
        f_value = np.mean(np.square(Y)) + lmbd * np.sum(E)
        params = np.array(E) * 0.
    return f_value, params


def data_gen(N ,P, sigma, seed=1):
    beta_true = np.array([1.]*10+[0]*(P-10))  
    rho = 0.
    def cov(P, rho = rho):
        V = np.empty([P,P])
        for i in range(P):
            for j in range(i):
                V[i,j] = np.power(rho, np.abs(i-j))
                V[j,i] = V[i,j]
        for i in range(P):
            V[i,i] = 1    
        return V
    V = cov(P)
    M = np.array([0]*P)
    ############
    np.random.seed(seed)
    ############
    X = np.random.multivariate_normal(mean=M, cov=V, size = N)
    noise = np.random.normal(scale = sigma, size = N)
    Y = np.matmul(X,beta_true) + noise
    Y = Y.astype(np.float32) 
    X = X.astype(np.float32) 
    return X, Y, V, beta_true



In [8]:
def main(ss):
    start = time.time()
    #Generate data
    X,Y,COV,beta_true = data_gen(args.N,args.P,args.sigma,seed=ss)   
    z_true = (beta_true>0)  
    phi = np.zeros(args.P)-2.0
    for i in range(1,args.iter):
        if args.grad == 'arm' or 'u2g' or 'reinforce':
            u_noise = np.random.uniform(0,1,[args.K,args.P])
            E1 = (u_noise>sigmoid(-phi)).astype(np.float32)
            E2 = (u_noise<sigmoid(phi)).astype(np.float32)
            Fs = []; 
            for j in range(args.K):
                if args.grad == 'arm' or 'u2g':
                    f1, _ = F(E1[j],X,Y, args.lmbd) 
                    f2, _ = F(E2[j],X,Y, args.lmbd)
                    Fs.append(f1 - f2)
                elif args.grad == 'reinforce' :
                    f2, _ = F(E2[j],X,Y, args.lmbd)
                    Fs.append(f2)
                else:
                    raise ValueError('No grad defined')
        if args.grad == 'arm':
            G = np.array(Fs)[:,None] * (u_noise - 0.5)
            mask = np.abs(E1-E2)
            G = G * mask
            grad = np.mean(G,axis=0)
        elif args.grad == 'u2g':
            G = np.array(Fs)[:,None] * (E1 - E2) * \
                np.maximum(sigmoid(phi),1-sigmoid(phi))[None,:] / 2.0
            grad = np.mean(G,axis=0)
        elif args.grad == 'reinforce':
            G = np.array(Fs)[:,None] * (E2-sigmoid(phi)[None,:]) # [K,1], 
            grad = np.mean(G,axis=0)
        else:
            raise ValueError('No grad defined')
            
        phi = phi - args.lr * grad
        probz = sigmoid(phi)
        entropy = np.sort(- probz * np.log(probz + eps))
        if np.mean(entropy[-args.P//100:])<0.15:
            break

    z_hat =  (sigmoid(phi) > 0.5) 
    _, beta_hat0 = F(z_hat,X,Y)
    beta_hat = np.zeros(args.P)
    if sum(z_hat)>eps:
        beta_hat[np.where(z_hat>0)] = beta_hat0   
    
    #Evaluation metrics
    ols_results = sm.OLS(Y, X).fit()
    beta_ols = ols_results.params
    
    TP = np.sum(z_true * z_hat) 
    FP = np.sum((1-z_true) * z_hat) 
    FN = np.sum(z_true * (1-z_hat)) 
    
    prec = TP/(TP + FP + eps)   
    rec = TP/(TP + FN + eps)
    F1 = 2 * prec * rec / (prec + rec + eps)     
    
    RTE_ols = 1 + np.matmul(np.matmul((beta_true-beta_ols), COV),\
                            (beta_true-beta_ols)) / (args.sigma**2)    
    RTE_g = 1 + np.squeeze(np.matmul(np.matmul((beta_hat-beta_true), COV),\
                            (beta_hat-beta_true).T) / (args.sigma**2))
    SNR = np.matmul(np.matmul(beta_true, COV),beta_true) / (args.sigma**2)         
    
    print('Trial-'+str(ss),': prec =', round(prec,2), 'rec =', round(rec,2), 'Iter =', i)
    
    corr = np.sum((1-z_true) * (1-z_hat))
    incorr = FN
    duration = time.time()-start

    all_ = [prec, rec, F1, corr, incorr, RTE_ols, \
            RTE_g, SNR, duration, args.lmbd]
    return(all_)

In [9]:
if __name__ == '__main__':
    start = time.time()
    pool = mp.Pool(mp.cpu_count()-1)
    results = pool.map(main, [i for i in range(args.N_test)])
    print('Time per trial is',(time.time()-start)/args.N_test)

Trial-6 : prec = 1.0 rec = 0.9 Iter = 2884
Trial-0 : prec = 1.0 rec = 1.0 Iter = 3170
Trial-2 : prec = 1.0 rec = 1.0 Iter = 3315
Trial-1 : prec = 1.0 rec = 1.0 Iter = 3630
Trial-4 : prec = 0.5 rec = 0.5 Iter = 6512
Trial-3 : prec = 0.54 rec = 0.7 Iter = 7808
Trial-7 : prec = 1.0 rec = 1.0 Iter = 3334
Trial-5 : prec = 0.6 rec = 0.6 Iter = 9115
Trial-8 : prec = 1.0 rec = 1.0 Iter = 4087
Trial-9 : prec = 1.0 rec = 1.0 Iter = 4813
Time per trial is 9.1713340044
